In [1]:
import tweepy
from tweepy import OAuthHandler
 
#use my API key to access twitter using tweepy
consumer_key = 'gWOM9f9FtTLgTrwarkVFVZohn'
consumer_secret = 'i14AQ4Dryxs0vuA4LalFPv2WykV1L3HbROrtfKWy6gwB1Q0EK2'
access_token = '109302020-RKTo6bMnFRSaX2EPbHBPnLLI7pkUplYPSEZVmOMV'
access_secret = 'uft6vpBXXVoL4YDpA9M2JZ91Z0tekZjwaEnYJcuDnNkv8'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

#page that lists NBA twitter feeds
url = "http://www.basketball-reference.com/friv/twitter.html"
    
#scrape the web page using BS
req = requests.get(url)
d = req.text
soup = BeautifulSoup(d, "lxml")

#get the table
table = soup.find_all('table')[0]

#blank dict
data = {
    'Player' : [],
    'Twitter' : [],
    'ID': []
}

#add player and twitter feed to dict
rows = table.find_all('tr')
for row in rows[2:]:
    r = row.find_all('a')
    #player and twitter feed
    data['Player'].append(r[0].get_text())
    data['Twitter'].append(r[1].get_text())
    #reg ex to extract palyer id
    temp = re.search('\/.\/(.+?)\.html', str(r[0]))
    data['ID'].append(temp.group(1))
    
#create a data frame
NBAtwitter = pd.DataFrame(data=data)
NBAtwitter.head()

,ID,Player,Twitter
0,acyqu01,Quincy Acy,QuincyAcy
1,adamsjo01,Jordan Adams,jordanadams1231
2,adamsst01,Steven Adams,RealStevenAdams
3,adebaba01,Bam Adebayo,Bam1of1
4,adrieje01,Jeff Adrien,Adrien4


In [3]:
#page that lists NBA player stats
url = "https://www.basketball-reference.com/leagues/NBA_2017_totals.html"
    
#scrape the web page using BS
req = requests.get(url)
d = req.text
soup = BeautifulSoup(d, "lxml")

#get the table
table = soup.find_all('table')[0]

#current players
currentNBA = []

rows = table.find_all('tr')
for row in rows[2:]:
    r = row.find_all('a')
    #skip header rows
    if len(r)==0:
        continue
    #reg ex to extract palyer id
    temp = re.search('\/.\/(.+?)\.html', str(r[0]))
    currentNBA.append(temp.group(1))
    
#remove duplicates
currentNBA = set(currentNBA)

#how many NBA players played in the league last season
len(currentNBA)

485

In [4]:
from __future__ import division

#only active NBA player twitter feeds
currentNBAtwitter = NBAtwitter[NBAtwitter['ID'].isin(currentNBA)].reset_index(drop=True)

#how many players have twitter feeds
round(len(currentNBAtwitter)/len(currentNBA), 3)

0.682

In [ ]:
#this is the code i wrote to get the friends for all NBA players
#you can now skip this part and take the csvs straight from my github

import time
import pandas as pd

#initialize a dataframe
df = pd.DataFrame(data={'src' : [], 'dst' : []})

index = -1
for each in currentNBAtwitter['Twitter']:
    #keep track of progress
    index += 1
    if index < 1000: #change this number to run
        continue
    #temp data
    g = {'src' : [], 'dst' : []}
    counter = 0
    ids = []
    #go through each page of followers for all our NBA twitter users
    for page in tweepy.Cursor(api.friends, screen_name=each, count=200).pages():
        ids.extend(page)
        print index, currentNBAtwitter['ID'][index], counter
        counter += 1
        #add the data 
        src = g['src'].extend([each for i in ids])
        dst = g['dst'].extend([user.screen_name for user in ids])
        #temp df
        df2 = pd.DataFrame(data=g)
        #save this to a csv
        df2.to_csv('data/nba_twitter/%s_friends.csv' % currentNBAtwitter['ID'][index])
    #add the temp data to the original
    df = df.append(df2, ignore_index=True)
    #save this to a csv - overwrite each time 
    #change number at the end every restart
    df.to_csv('data/nba_twitter/all_nba/nba_twitter_friends.csv')

In [ ]:
#read csv and add to pd data frame - be careful 12m rows
base = "https://raw.githubusercontent.com/capstat/MSDA_620/master/data/nba_twitter/nba_twitter_friends%s.csv"

#initialize a dataframe
df = pd.DataFrame(data={'src' : [], 'dst' : []})

for i in range(1, 41):
    temp = pd.read_csv(base % i)
    #drop useless index row
    del temp['X']
    df = df.append(temp, ignore_index=True)

print len(df)

In [ ]:
#use networkx for calculations
import networkx as nx

g = nx.from_pandas_dataframe(df, source='src', target='dst')

In [ ]:
#network too big - trim
pop = []
for each in g.nodes():
    if len(g.neighbors(each)) > 20:
        pop.append(each)

print len(pop)

In [ ]:
df2 = df[df['dst'].isin(pop)]
len(df2)

In [ ]:
#twitter feeds with more than 20 neighbors
ns = [x[0] for x in cc]
not_player = set(ns) - set(currentNBAtwitter['Twitter'])

In [ ]:
b = nx.closeness_centrality(gg)
bb = sorted(b.iteritems(), key=lambda (k,v): (-v,k))
bbb = [t for t in bb if t[0] in not_player]

print "Betweeness Centrality"
for each in bbb[:10]: print each[0], ": ", each[1]

In [ ]:
b = nx.betweenness_centrality(gg)
bb = sorted(b.iteritems(), key=lambda (k,v): (-v,k))
bbb = [t for t in bb if t[0] in not_player]

print "Betweeness Centrality"
for each in bbb[:10]: print each[0], ": ", each[1]

In [ ]:
b = nx.pagerank(gg)
bb = sorted(b.iteritems(), key=lambda (k,v): (-v,k))
bbb = [t for t in bb if t[0] in not_player]

print "Betweeness Centrality"
for each in bbb[:10]: print each[0], ": ", each[1]